In [47]:
import requests
from bs4 import BeautifulSoup
import json

In [3]:
url = 'https://www.meesho.com/stylish-casual-block-heels-for-women/p/5y67an'

In [49]:
response = requests.get(url)
soup = BeautifulSoup(response.content,'html.parser')


In [23]:
scripts = soup.find_all('script', type = 'application/ld+json')
scripts

[<script type="application/ld+json">{"@context":"https://schema.org","@type":"Organization","url":"https://www.meesho.com","logo":"https://www.meesho.com/assets/svgicons/meeshoLogo.svg","name":"Meesho","contactPoint":[{"@type":"ContactPoint","email":"query@meesho.com","contactType":"Customer Service"}],"sameAs":["https://x.com/Meesho_Official","https://www.facebook.com/meeshosupply","https://www.instagram.com/meeshoapp","https://www.youtube.com/channel/UCaGHIRKYUYlaI_ZAt2hxpjw","https://www.linkedin.com/company/meesho"]}</script>,
 <script type="application/ld+json">{"@context":"http://schema.org","@type":"Product","name":"Stylish Casual Block heels for women","image":["https://images.meesho.com/images/products/359727215/bqmlz_512.webp","https://images.meesho.com/images/products/359727215/lapwh_512.webp"],"sku":"5y67an","mpn":"5y67an","brand":{"@type":"Thing","name":"Deva Ji Traders"},"offers":{"@type":"Offer","priceCurrency":"INR","price":249,"availability":"OutOfStock","url":"https:/

In [25]:
for script in scripts:
    try:
        data = json.loads(script.string)
        if "review" in data:
            for review in data["review"]:
                print(f"Customer Name : {review['author']['name']}")
                print(f"Rating : {review['reviewRating']['ratingValue']}")
                print(f"Review : {review['reviewBody']}")
                print(f"Tag : Not available in static HTML")
    except Exception as e:
        continue

Customer Name : Mangla Bopche
Rating : 5
Review : It's very classy and aesthetic💗. Perfect fit and colour💗
Tag : Not available in static HTML
Customer Name : Ruhy Nasim 
Rating : 5
Review : All is fine but just doesn't look that much satisfying as picture . Rest is well. In 257 is ok. Facing only one issue that it's HEELS ticking sound
Tag : Not available in static HTML


In [27]:
rating_tag = soup.find('h1',{'class' : 'sc-eDvSVe cdZTwf'})
print("Overall rating:",rating_tag.get_text())

Overall rating: 4.1


In [29]:
# --- Additional Features Below (without changing original) ---

# Extract product name and price from structured data
for script in scripts:
    try:
        data = json.loads(script.string)
        if data.get("@type") == "Product":
            print("Product Name:", data.get("name"))
            
            # Extract price
            price = data.get("offers", {}).get("price")
            currency = data.get("offers", {}).get("priceCurrency")
            if price:
                print("Price:", f"{currency} {price}")
            break
    except:
        continue


Product Name: Stylish Casual Block heels for women
Price: INR 249


In [35]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

# Base URL
base_url = 'http://books.toscrape.com/catalogue/page-{}.html'

# Lists to store data
titles = []
prices = []
availabilities = []
ratings = []

# Loop through the first 5 pages
for page in range(1, 6):
    url = base_url.format(page)
    response = requests.get(url)
    if response.status_code != 200:
        print(f"Failed to retrieve page {page}")
        continue

    soup = BeautifulSoup(response.text, 'html.parser')
    books = soup.find_all('article', class_='product_pod')

    for book in books:
        # Title
        title = book.h3.a['title']
        titles.append(title)

        # Price
        price = book.find('p', class_='price_color').text.strip()
        prices.append(price)

        # Availability
        availability = book.find('p', class_='instock availability').text.strip()
        availabilities.append(availability)

        # Rating
        rating_class = book.p['class']
        rating = rating_class[1] if len(rating_class) > 1 else 'No rating'
        ratings.append(rating)

# Create DataFrame
df = pd.DataFrame({
    'Title': titles,
    'Price': prices,
    'Availability': availabilities,
    'Rating': ratings
})

# Display the first few rows
print(df.head())


                                   Title    Price Availability Rating
0                   A Light in the Attic  Â£51.77     In stock  Three
1                     Tipping the Velvet  Â£53.74     In stock    One
2                             Soumission  Â£50.10     In stock    One
3                          Sharp Objects  Â£47.82     In stock   Four
4  Sapiens: A Brief History of Humankind  Â£54.23     In stock   Five


In [39]:
df['Price'] = df['Price'].str.replace('Â£', '').astype(float)


In [41]:
in_stock = df['Availability'].str.contains('In stock').sum()
print(f"Books in stock: {in_stock}")


Books in stock: 100


In [43]:
rating_counts = df['Rating'].value_counts()
print(rating_counts)


Rating
Three    22
One      22
Five     19
Two      19
Four     18
Name: count, dtype: int64
